In [1]:
using Rocket
using BenchmarkTools

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1278


In [2]:
g = (x) -> ForwardDiff.gradient((x) -> det(inv(cholesky(x))), x)
f = (x) -> ForwardDiff.gradient((x) -> det(inv(cholesky(Positive, x))), x)

#9 (generic function with 1 method)

In [25]:
@benchmark collectLatest(s) setup=(s=[ of(1), from([ 1, 2 ]) ]) 

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.709 ns (0.00% GC)
  median time:      1.777 ns (0.00% GC)
  mean time:        1.822 ns (0.00% GC)
  maximum time:     6.699 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [26]:
@code_lowered Rocket.union_type([ of(1), from([ 1, 2 ]) ])

CodeInfo(
1 ─     return $(Expr(:static_parameter, 1))
)

In [23]:
@btime Rocket.union_type(s) setup=(s=[ of(1), of(2) ])

  0.036 ns (0 allocations: 0 bytes)


Int64

In [3]:
collected = collectLatest([ of(1) |> async(0), from([ 1, 2 ]) |> async(0) ])

subscribe!(collected, logger())

[LogActor] Data: [1, 1]


CollectLatestSubscription()

[LogActor] Data: [1, 2]
[LogActor] Completed


In [11]:
latest = combineLatest(of(1) |> async(0), from(1:5) |> async(0))

subscribe!(latest, logger())

[LogActor] Data: (1, 1)


CombineLatestSubscription()

[LogActor] Data: (1, 2)
[LogActor] Data: (1, 3)
[LogActor] Data: (1, 4)
[LogActor] Data: (1, 5)
[LogActor] Completed


In [17]:
subscribe!(collectLatest([ from(1:5), of(2.0) ]), logger())

[LogActor] Data: Union{Float64, Int64}[5, 2.0]
[LogActor] Completed


CollectLatestSubscription()

In [19]:
?isdefined(r, 1)

```
isdefined(m::Module, s::Symbol)
isdefined(object, s::Symbol)
isdefined(object, index::Int)
```

Tests whether a global variable or object field is defined. The arguments can be a module and a symbol or a composite object and field name (as a symbol) or index.

To test whether an array element is defined, use [`isassigned`](@ref) instead.

See also [`@isdefined`](@ref).

# Examples

```jldoctest
julia> isdefined(Base, :sum)
true

julia> isdefined(Base, :NonExistentMethod)
false

julia> a = 1//2;

julia> isdefined(a, 2)
true

julia> isdefined(a, 3)
false

julia> isdefined(a, :num)
true

julia> isdefined(a, :numerator)
false
```


In [32]:
collected = collectLatest([ of(1), from([ 1, 2 ]) ], sum)

subscribe!(collected, logger())

LoadError: Actor of type LoggerActor{Array{Int64,1},Nothing} expects data to be of type Array{Int64,1}, but data of type Int64 has been found.

In [43]:
function foo(x)
    @show typeof(sign(x))
    return sign(x)
end

foo (generic function with 1 method)

In [49]:
ForwardDiff.derivative(foo, 0.0)

typeof(sign(x)) = ForwardDiff.Dual{ForwardDiff.Tag{typeof(foo),Float64},Float64,1}


1.0